In [1]:
import datasets
import dateutil
from obfuscation.fake_date_enumerate import FakeDate
from datetime import datetime
import time

c:\Users\zhma\Projects\deid\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
wfu_dataset = datasets.load_dataset('wfudata', trust_remote_code=True)

In [3]:
fakedate = FakeDate()

In [4]:
util_timer = 0.0
my_timer = 0.0

for i in range(len(wfu_dataset['train'])):
    phi = wfu_dataset['train']['phi'][i]
    text = wfu_dataset['train']['text'][i]
    prev_offset = None
    for phi_type, offset in zip(phi['types'], phi['offsets']):
        if phi_type == 'DATE':
            # see if need to stick things before this tag
            if prev_offset is not None and offset[0] == prev_offset[1] + 1:
                offset[0] = prev_offset[0]
            date_text = text[offset[0]:offset[1]]

            t1 = time.perf_counter()
            try:
                util_parsed = dateutil.parser.parse(date_text, 
                                                    default=datetime(2024,1,1,0,0))
            except:
                util_parsed = None
            util_timer += time.perf_counter() - t1

            t1 = time.perf_counter()
            dt_date, dt_time, _, _ = fakedate.parse(date_text)
            my_timer += time.perf_counter() - t1

            # for comparision, need to stick dt_date and dt_time
            # default to 1/1/2024
            if dt_date is None:
                dt_date = datetime(2024,1,1,0,0)         
            if dt_date.year == 1900:
                dt_date = dt_date.replace(year=2024)
            if dt_time is not None:
                dt_date = dt_date.replace(hour=dt_time.hour, 
                                          minute=dt_time.minute,
                                          second=dt_time.second)

            if util_parsed is None or dt_date is None or util_parsed != dt_date:
                print(f' ---- {date_text} ---- ')
                print(util_parsed)
                print(dt_date)

            prev_offset = offset[:]

 ---- 1711219 ---- 
None
2024-01-01 00:00:00
 ---- 09/01/2018 8/31/2018 ---- 
None
2018-09-01 00:00:00
 ---- 08/31/2020 8/31/2017 ---- 
None
2020-08-31 00:00:00
 ---- 09/01/2018 8/31/2018 ---- 
None
2018-09-01 00:00:00
 ---- 08/31/2020 8/31/2017 ---- 
None
2020-08-31 00:00:00
 ---- 06/20/12Electronic ---- 
None
2024-01-01 00:00:00
 ---- 09/27/13Electronic ---- 
None
2024-01-01 00:00:00
 ---- 2012-13 ---- 
None
2024-01-01 00:00:00
 ---- 2/19/2013 2/19/2013 ---- 
None
2013-02-19 00:00:00
 ---- 2/19/2013 2/19/2013 2/19/2013 ---- 
None
2024-01-01 00:00:00
 ---- 2/19/2013 2/19/2013 2/19/2013 2/19/2013 ---- 
None
2024-01-01 00:00:00
 ---- 03/08/2018 3/8/2017 ---- 
None
2018-03-08 00:00:00
 ---- 04/17/2018 4/17/2008 ---- 
None
2018-04-17 00:00:00
 ---- 08/01/2018 3/6/2018 ---- 
None
2018-08-01 00:00:00
 ---- 04/19/18 135/92 ---- 
None
2018-04-19 00:00:00
 ---- 155/101 ---- 
None
2024-01-01 00:00:00
 ---- MD10/22/2012 10:28:35 ---- 
None
2024-01-01 10:28:35
 ---- 10/12/2017 10/12/2016 ---- 
No

In [5]:
util_timer, my_timer

(0.5686287892749533, 0.7825161053333431)